# Stage 1: Generate Stock Universe

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Gather price histories

In [16]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---5311'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

In [3]:
td_ameritrade = amc.AmeritradeRest(username, password, client_id, chrome_executabel_path)
td_ameritrade.authenticate()

if len(td_ameritrade.authorization) == 0:
    print('Error: No authorization data: {}'.format(td_ameritrade.authorization))
else:
    print('You have authorization')

You have authorization


## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

## First lets see why stocks we already own for a specific account

I only want to work with Equity investments. This is kind of confusing, but at the account level assets that can be traded are call "EQUITY". When you get quotes for each asset, the same asset can be something like "ETF".

I also use Ameritrade's portfolio planner tool to create an asset mix based off of their reccomendations. I don't want these stocks (or in my case mutual funds and ETFs) to be part of this analysis. So I'll remove them here.

In [4]:
# Specific Portfolio Account
account_portfolio_df = utils.get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
equity_investments_df = utils.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')

# Filter out non Equity investments
current_stocks = amc.AmeritradeRest(username, password, client_id).get_quotes(utils.get_investment_symbols(equity_investments_df)).query('assetType == "EQUITY"').index.tolist()
current_investments_df = equity_investments_df[equity_investments_df['symbol'].isin(current_stocks)]
current_investments_df

,account,shortQuantity,averagePrice,currentDayProfitLoss,currentDayProfitLossPercentage,longQuantity,settledLongQuantity,settledShortQuantity,marketValue,maintenanceRequirement,currentDayCost,previousSessionLongQuantity,assetType,cusip,symbol,description,type
19,#---5311,0.0,21.62526,2.000000e-12,0.0,783.0,783.0,0.0,12661.11,12661.11,0.0,783.0,EQUITY,88688T100,TLRY,0,0
26,#---5311,0.0,0.14474,0.000000e+00,0.0,45000.0,45000.0,0.0,8673.75,0.00,0.0,45000.0,EQUITY,Q3860H107,FGPHF,0,0
27,#---5311,0.0,13.74495,0.000000e+00,0.0,2000.0,2000.0,0.0,25960.00,25960.00,0.0,2000.0,EQUITY,98138J206,WKHS,0,0
28,#---5311,0.0,0.04960,0.000000e+00,0.0,250.0,250.0,0.0,6.75,0.00,0.0,250.0,EQUITY,Q3394D101,EEENF,0,0
29,#---5311,0.0,59.22000,0.000000e+00,0.0,50.0,50.0,0.0,2454.50,2454.50,0.0,50.0,EQUITY,26142R104,DKNG,0,0


## Remove other assets

There may be some stocks that you are speculating on and do not want to be part of the analysis. Being a conservative investor, I have a percentage of my active portfolio (that is not part of the portfolio planner), that I have personally speculated on and are using for a long term play. These stocks will not be part of the portfolio optimization.

In [5]:
speculative_stocks = ['FGPHF', 'EEENF']
final_investments_df = current_investments_df[~current_investments_df['symbol'].isin(speculative_stocks)]
final_existing_stocks = utils.get_investment_symbols(final_investments_df)
final_existing_stocks 

['TLRY', 'WKHS', 'DKNG']

In [6]:
symbols_of_interest = ['MGM', 'PDYPF', 'NNXPF', 'WKHS']
# Hardcoded for now
symbols_via_specific_criteria = ['AAPL',
                                 'MSFT',
                                 'GOOG',
                                 'TSLA',
                                 'COKE',
                                 'IBM',
                                 'BABA',
                                 'GMGMF',
                                 'OEG',
                                 'LX',
                                 'AIH',
                                 'NMRD',
                                 'CAN',
                                 'MOSY',
                                 'QFIN',
                                 'OCG',
                                 'PRTK',
                                 'ZKIN', 
                                 'EFOI',
                                 'CONN',
                                 'LEDS',
                                 'TELL',
                                 'JZXN',
                                 'VTNR',
                                 'AEI',
                                 'RCON'
                                ]

In [7]:
snp_500_df = utils.get_snp500()
display(snp_500_df.head())
snp_500_symbols = snp_500_df.index.to_list()
quote_dfs = []
for i in range(0, 500, 100):
    quote_dfs.append(amc.AmeritradeRest(username, password, client_id).get_quotes(snp_500_symbols[i:i+100]))
    
snp_500_quotes_df = pd.concat(quote_dfs, axis=0)
snp_500_quotes_df.describe()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago , Illinois",1964-03-31,0000001800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago , Illinois",2012-12-31,0001551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin , Ireland",2011-07-06,0001467373,1989


,bidPrice,bidSize,askPrice,askSize,lastPrice,lastSize,openPrice,highPrice,lowPrice,closePrice,...,divAmount,divYield,regularMarketLastPrice,regularMarketLastSize,regularMarketNetChange,regularMarketTradeTimeInLong,netPercentChangeInDouble,markChangeInDouble,markPercentChangeInDouble,regularMarketPercentChangeInDouble
count,500.000000,500.000000,500.000000,500.000000,500.000000,5.000000e+02,500.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.0,5.000000e+02,500.000000,500.000000,500.0,500.0
mean,188.548240,400.600000,207.577480,430.000000,198.946002,8.941860e+04,198.011562,200.000027,196.650197,199.007440,...,1.770110,1.588240,199.007440,4066.986000,0.0,1.625868e+12,0.012600,0.000000,0.0,0.0
std,363.469425,1779.807594,395.575089,1109.053651,369.255673,1.860470e+05,370.087660,373.215224,366.949606,369.491113,...,1.937324,1.452237,369.491113,6536.500513,0.0,4.922404e+06,0.324705,0.000011,0.0,0.0
min,9.800000,0.000000,11.510000,0.000000,11.510000,0.000000e+00,11.510000,11.530000,11.405000,11.510000,...,0.000000,0.000000,11.510000,1.000000,0.0,1.625861e+12,-1.942000,-0.000100,0.0,0.0
25%,57.997500,100.000000,61.275000,100.000000,60.157500,0.000000e+00,59.722500,60.367500,59.460000,60.022500,...,0.175000,0.195000,60.022500,1097.000000,0.0,1.625861e+12,0.000000,0.000000,0.0,0.0
50%,108.575000,100.000000,119.405000,100.000000,115.885000,5.000000e+01,115.095000,116.565000,114.405000,115.885000,...,1.360000,1.340000,115.885000,2141.000000,0.0,1.625872e+12,0.000000,0.000000,0.0,0.0
75%,200.687500,200.000000,221.825000,400.000000,214.996250,1.150500e+05,214.392500,215.523125,213.102500,214.795000,...,2.640000,2.500000,214.795000,4362.750000,0.0,1.625872e+12,0.006825,0.000000,0.0,0.0
max,5022.220200,30700.000000,5900.000000,13600.000000,5060.839800,1.899100e+06,5114.000000,5161.778300,5050.000000,5060.839800,...,16.520000,7.400000,5060.839800,67511.000000,0.0,1.625875e+12,2.439700,0.000200,0.0,0.0


In [23]:
snp_500_quotes_df.columns

Index(['assetType', 'assetMainType', 'cusip', 'symbol', 'description',
       'bidPrice', 'bidSize', 'bidId', 'askPrice', 'askSize', 'askId',
       'lastPrice', 'lastSize', 'lastId', 'openPrice', 'highPrice', 'lowPrice',
       'bidTick', 'closePrice', 'netChange', 'totalVolume', 'quoteTimeInLong',
       'tradeTimeInLong', 'mark', 'exchange', 'exchangeName', 'marginable',
       'shortable', 'volatility', 'digits', '52WkHigh', '52WkLow', 'nAV',
       'peRatio', 'divAmount', 'divYield', 'divDate', 'securityStatus',
       'regularMarketLastPrice', 'regularMarketLastSize',
       'regularMarketNetChange', 'regularMarketTradeTimeInLong',
       'netPercentChangeInDouble', 'markChangeInDouble',
       'markPercentChangeInDouble', 'regularMarketPercentChangeInDouble',
       'delayed', 'realtimeEntitled', 'market_cap'],
      dtype='object')

In [8]:
snp_500_tickers = snp_500_quotes_df.index.to_list()

In [9]:
stock_universe = set(symbols_of_interest + symbols_via_specific_criteria + final_existing_stocks + snp_500_tickers)
holdings = utils.get_holdings(final_investments_df, stock_universe)['marketValue']
display(holdings)
utils.save_port_data(holdings.reset_index(), portfolio_file_name)

symbol
A       0.0
AAL     0.0
AAP     0.0
AAPL    0.0
ABBV    0.0
       ... 
XLNX    0.0
XOM     0.0
XRAY    0.0
XYL     0.0
ZKIN    0.0
Name: marketValue, Length: 527, dtype: float64

## Portfolio weights

With the portfolio stocks and the additional stocks, show how each of them contribute to the portfolio. Later, once we produce an optimized portfolio, we can generate a report on how much stock to buy/sell based of what we already have.

In [10]:
holding_weights = utils.get_portfolio_weights(holdings)
display(holding_weights)

symbol
A       0.0
AAL     0.0
AAP     0.0
AAPL    0.0
ABBV    0.0
       ... 
XLNX    0.0
XOM     0.0
XRAY    0.0
XYL     0.0
ZKIN    0.0
Length: 527, dtype: float64

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 3 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [14]:
def get_price_histories(tickers, end_date=None, num_periods=1):
    price_histories_df = pd.DataFrame()
    ticker_count = 0
    for symbol in tqdm(tickers, desc='Tickers', unit='Price Histories'):
        ticker_price_history = td_ameritrade.get_daily_price_history(symbol, end_date, num_periods=num_periods)
        if ticker_price_history is not None:
            price_histories_df = price_histories_df.append([ticker_price_history])
        ticker_count += 1
        if ticker_count % 30 == 0:
            time.sleep(10)
    price_histories_df.reset_index(drop=True, inplace=True)
    return price_histories_df.sort_values(by=['date'])

number_of_years = 3
price_histories = get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)

Tickers:   0%|          | 0/527 [00:00<?, ?Price Histories/s]

Empty candle data for IPQ


In [17]:
number_of_years = 3
price_histories = amc.AmeritradeRest(username, password, client_id).get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Tickers:   0%|          | 0/527 [00:00<?, ?Price Histories/s]

Empty candle data for IPQ


In [18]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,232.39,232.39,226.855,229.64,1002146,PSA,2018-07-09
381952,52.04,53.31,52.010,53.29,2231829,HIG,2018-07-09
184885,77.26,78.35,76.890,77.94,660417,ALLE,2018-07-09
16191,47.69,47.71,46.190,46.37,4564154,SO,2018-07-09
184128,122.76,124.92,122.430,124.48,634371,JBHT,2018-07-09


In [19]:
price_histories = utils.read_price_histories(price_histories_file_name)
close = utils.get_close_values(price_histories)
close.tail()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL,ZKIN
date,,,,,,,,,,,,,,,,,,,,,
2021-07-02 00:00:00+00:00,148.82,21.48,211.05,139.96,115.17,116.62,317.02,118.04,304.84,593.07,...,365.74,23.26,34.31,121.50,66.82,144.56,63.17,63.89,120.89,3.9500
2021-07-06 00:00:00+00:00,148.98,21.01,208.70,142.02,115.73,115.42,326.71,118.82,305.40,596.90,...,368.99,22.96,33.97,116.90,67.07,144.17,61.37,63.09,120.46,3.9300
2021-07-07 00:00:00+00:00,149.49,20.31,209.44,144.57,116.75,115.04,334.53,119.87,309.61,605.77,...,372.66,23.23,34.84,112.91,67.69,137.44,60.41,63.63,120.67,3.6200
2021-07-08 00:00:00+00:00,148.83,20.35,207.51,143.24,116.35,114.44,325.04,119.26,309.15,605.95,...,373.11,22.94,35.04,111.66,67.50,133.77,60.14,61.91,119.19,3.6600
2021-07-09 00:00:00+00:00,150.03,20.89,213.47,145.11,116.58,115.13,324.77,119.74,312.62,604.50,...,374.19,23.31,35.82,113.87,67.67,135.59,61.23,62.42,121.05,3.7181
